基礎データの出力

In [1]:
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import math
import numpy as np
import os
import collections

In [2]:
folder = "puyopuyo_saikyo_league/stats/s1_day1_momoken_rein_2"
games = 57

In [3]:
#勝利数を計算
wins_1P = 0
wins_2P = 0

for i in range(games):
    #ファイル読み込み
    filepath = folder + "/" + str(i) + "_6.csv"
    df_game = pd.read_csv(filepath)
    #勝利すれば+1
    if df_game["Win"].iloc[0] == 1:
        wins_1P += 1
    else:
        wins_2P += 1

print(wins_1P, wins_2P)

30 27


In [4]:
#全消し戦の勝利数を計算
wins_zenkeshi_1P = 0
wins_zenkeshi_2P = 0

for i in range(games):
    #ファイル読み込み
    filepath = folder + "/" + str(i) + "_6.csv"
    df_game = pd.read_csv(filepath)
    #全消し戦で勝利すれば+1
    if df_game["Zenkeshi"].iloc[0] == 1:
        if df_game["Win"].iloc[0] == 1:
            wins_zenkeshi_1P += 1
        else:
            wins_zenkeshi_2P += 1

print(wins_zenkeshi_1P, wins_zenkeshi_2P)

5 5


In [5]:
#連鎖数を集計
#連鎖数分布を出力
rensa_list_1P = []
rensa_list_2P = []

for i in range(games):
    #ファイル読み込み
    filepath = folder + "/" + str(i) + "_6.csv"
    df_game = pd.read_csv(filepath)
    N = len(df_game["Frame"])
    
    for j in range(N):            
        #連鎖がない試合はスキップ
        if np.isnan(df_game["Rensa"].iloc[j]):
            pass
        #プレイヤー毎にrensa_list_nPに格納
        else:
            if (df_game["Player"].iloc[j] == 1):
                rensa_list_1P.append(df_game["Rensa"].iloc[j])
            else:
                rensa_list_2P.append(df_game["Rensa"].iloc[j]) 

rensa_times_1P = []
rensa_times_2P = []
rensa_index = []

#DataFrameに出力                
for i in range(1, 20):
    rensa_times_1P.append(rensa_list_1P.count(i))
    rensa_times_2P.append(rensa_list_2P.count(i))
    rensa_index.append(i)

df_rensa = pd.DataFrame(list(zip(rensa_index, rensa_times_1P, rensa_times_2P)), columns=["連鎖数", "1P回数", "2P回数"])    
df_rensa

,連鎖数,1P回数,2P回数
0,1,69,55
1,2,19,29
2,3,16,18
3,4,10,7
4,5,9,11
5,6,11,8
6,7,2,7
7,8,8,2
8,9,2,5
9,10,3,3


In [6]:
#回収率出力
#本線(≠セカンド)に絞る
#初期化
kaishu_list_1P = []
kaishu_list_2P = []
kaishu_list_1P_2 = []
kaishu_list_2P_2 = []

for i in range(games):
    #データ読み込み
    filepath = folder + "/" + str(i) + "_6.csv"
    df_game = pd.read_csv(filepath)
    N = len(df_game["Frame"])
    
    #初期化
    tmp_1P = 0
    tmp_2P = 0

    for j in range(N):
        if np.isnan(df_game["Kaishu"].iloc[j]):
            pass
        #試合で最初に発火した本線に限り回収率をkaishu_listに格納
        else:
            if (df_game["Player"].iloc[j]*df_game["Honsen"].iloc[j] == 1) & (tmp_1P == 0):
                kaishu_list_1P.append(df_game["Kaishu"].iloc[j])
                tmp_1P = 1
            elif (df_game["Player"].iloc[j]*df_game["Honsen"].iloc[j] == 2) & (tmp_2P == 0):
                kaishu_list_2P.append(df_game["Kaishu"].iloc[j])
                tmp_2P = 1
            else:
                pass          

#(誤差が大きいときに使用→回収率50%未満、100%より上はエラーとしてカウントしない)
for i in kaishu_list_1P:
    if (50<= i <= 100):
        kaishu_list_1P_2.append(i)
        
for i in kaishu_list_2P:
    if (50 <= i <= 100):
        kaishu_list_2P_2.append(i)


In [28]:
#基礎データ出力
win_list = [wins_1P, wins_2P]
zenkeshi_list = [wins_zenkeshi_1P, wins_zenkeshi_2P]
max_rensa_list = [max(rensa_list_1P), max(rensa_list_2P)]
kaishu_median_list = [round(np.median(kaishu_list_1P_2),1), round(np.median(kaishu_list_2P_2),1)]
                 
df_basic_data = pd.DataFrame([win_list, zenkeshi_list, max_rensa_list, kaishu_median_list],
                             index=["勝利数", "全消し戦勝利数", "最大連鎖数", "回収率の中央値"], columns=["1P", "2P"]  )    
df_basic_data

,1P,2P
勝利数,30.0,27.0
全消し戦勝利数,5.0,5.0
最大連鎖数,13.0,15.0
回収率の中央値,88.2,86.8
